In [2]:
import pandas as pd
from urllib.request import urlopen
import requests
import re
from bs4 import BeautifulSoup
from IPython.display import display, HTML
import glob

# 1. Obtención del dataset a partir de los HTML con los anuncios de empleo

1. Cargar ficheros HTML

Empezamos cargando todos los archivos HTML de los directorios **jobs_ca**, **jobs_uk** y **jobs_usa**. Almacenamos el contenido de estos archivos en una lista **html_files**.

In [3]:
html_files = []

for file in sorted(glob.glob('jobs_uk/*.html')):
    with open(file, 'r', encoding='utf-8') as f:
        html_files.append(f.read())
        
for file in sorted(glob.glob('jobs_usa/*.html')):
    with open(file, 'r', encoding='utf-8') as f:
        html_files.append(f.read())
        
for file in sorted(glob.glob('jobs_ca/*.html')):
    with open(file, 'r', encoding='utf-8') as f:
        html_files.append(f.read())         

Utilizamos el módulo **glob** de Python 3 para obtener los nombres de los archivos con extensiones HTML en los directorios **jobs_ca**, **jobs_uk** y **jobs_usa**. Estos nombres de archivo se ordenan para mantener la coherencia de la salida en las distintas máquinas personales de los lectores. Esto garantiza que los dos primeros archivos de la muestra sean los mismos para todos los lectores.

In [4]:
print(f"{len(html_files)} archivos HTML cargados.")

4308 archivos HTML cargados.


Cada uno de nuestros 1458 archivos HTML puede ser analizado utilizando **Beautiful Soup**. Ejecutamos el análisis y almacenamos los resultados en una lista **soup_htmls**. También verificamos que cada archivo HTML analizado incluya un título y un cuerpo.

2. Analizando los archivos HTML

In [5]:
soup_htmls = []
for html in html_files:
    soup = BeautifulSoup(html)
    assert soup.title is not None
    assert soup.body is not None
    soup_htmls.append(soup)

3. Crear un DataFrame cuyas columnas contengan el título y el cuerpo de cada HTML

In [6]:
table_dict = {'title_html': [], 'body_html': []}

for soup in soup_htmls:
    title = soup.find('title').text.lstrip().rstrip()
    body = soup.find('body').text.lstrip().rstrip()
    table_dict['title_html'].append(title)
    table_dict['body_html'].append(body)

In [7]:
df = pd.DataFrame(table_dict)
df.describe()

,title_html,body_html
count,4308,4308
unique,3889,3929
top,"Functional Analyst, HRMS Job in Edmonton for S...",Product Specialist Description JOB SUMMARYThe...
freq,9,23


3889 de los 4308 títulos (title\_html) son únicos, los 419 títulos restantes son duplicados. El título más común se repite 9 veces: es para un puesto de **Analista funcional**. Por otro lado, 3929 cuerpos (body\_html) son únicos, por lo que 379 son duplicados. Por lo tanto, debemos eleiminar los duplicados de la columna body\_html aunque algunas ofertas compartan un título genérico común.

## 1.1. Exploración del HTML para la descripción de competencias

Comenzamos nuestra exploración renderizando el HTML con el índice 0 de **html_files**.

1. Representación del HTML de la primera oferta de empleo

In [8]:
display(HTML(html_files[0]))

HTML renderizado de la primera oferta de empleo. El párrafo inicial resume el puesto de ciencia de datos. El párrafo va seguido de una lista de viñetas, cada una de las cuales contiene una habilidad necesaria para conseguir el puesto.


La oferta de empleo es para un puesto en ciencia de datos. El anuncio comienza con una breve descripción del puesto, de la que se desprende que el trabajo consiste en extraer información de los datos gubernamentales. Entre las competencias requeridas figuran la creación de modelos, la estadística y la visualización. Estas competencias se detallan en las dos subsecciones en negrita: Responsabilidades y Cualificaciones. Cada subapartado se compone de varias viñetas de una sola frase. El contenido de las viñetas es variado: las responsabilidades incluyen el uso de métodos estadísticos (viñeta 1), el descubrimiento de tendencias futuras (viñeta 5), el análisis espacial de datos geográficos (viñeta 6) y la visualización estéticamente atractiva (viñeta 7). Además, las cualificaciones indicadas en las viñetas abarcan lenguajes informáticos como R o Python (viñeta 1), herramientas de visualización como Matplotlib (viñeta 2), técnicas de aprendizaje automático, incluida la agrupación (viñeta 3), y conocimientos de conceptos estadísticos avanzados (viñeta 4).

Cabe señalar que las cualificaciones no difieren mucho de las responsabilidades. Sí, las cualificaciones se centran en las herramientas y los conceptos, mientras que las responsabilidades están más orientadas a las acciones en los puestos de trabajo; pero en cierto modo, sus viñetas son intercambiables. Cada viñeta describe una habilidad que el candidato debe tener para desempeñar bien el trabajo. Así, podemos subdividir html_files[0] en dos partes conceptualmente diferentes:
   * Un resumen inicial del puesto
   * Una lista con viñetas de las competencias necesarias para conseguir el empleo

¿La siguiente oferta de empleo está estructurada de forma similar? Lo averiguaremos renderizando html_files[1] (figura 2).

4. Renderizado del HTML de la segunda oferta de empleo

In [9]:
display(HTML(html_files[1]))

HTML renderizado de la segunda oferta de empleo. Al igual que en el primer anuncio, el párrafo inicial resume el puesto de ciencia de datos y una lista de viñetas describe las habilidades necesarias para conseguirlo.

La oferta de empleo es para un puesto de ciencia de datos en una empresa de marketing de inteligencia artificial. La estructura del anuncio es similar a la de **html_files[0]**: el puesto se resume en el párrafo inicial del anuncio y, a continuación, se presentan las competencias requeridas en viñetas. Estas competencias en viñetas son variadas en cuanto a requisitos técnicos y detalles. Por ejemplo, la cuarta viñeta desde abajo exige experiencia en la pila de ciencia de datos de Python (NumPy, SciPy, Pandas, scikit-learn), la siguiente viñeta requiere un historial de resolución de problemas empresariales difíciles en el mundo real, y la última viñeta exige excelentes habilidades de comunicación escrita y verbal. Estas tres habilidades son muy diferentes. La diferencia es intencionada: el autor del anuncio hace hincapié en los diversos requisitos necesarios para obtener el puesto. Así pues, las viñetas de **html_files[0]** y **html_files[1]** tienen un único propósito: ofrecernos descripciones breves, en forma de frase, de las habilidades únicas que se requieren para cada puesto.

¿Aparecen este tipo de descripciones con viñetas en otras ofertas de empleo? Vamos a averiguarlo. Primero extraeremos las viñetas de cada uno de nuestros archivos HTML analizados.  Como recordatorio, una viñeta está representada por la etiqueta HTML \<li\>. Así, podemos extraer una lista de viñetas de un objeto soup llamando a **soup.find_all('li')**. A continuación, iteraremos sobre nuestra lista soup_objects y extraeremos todas las viñetas de cada elemento de esa lista. Almacenamos estos resultados en una columna **li_body** de nuestra tabla **df** existente.

5. Extraer viñetas del HTML

In [10]:
df['li_body'] = [[li.text.strip() for li in soup.find_all('li')] for soup in soup_htmls]
#df['Bullets'][2]

6. Extraer contenido del **body** sin viñetas del HTML

In [11]:
unlisted_body = []
for soup in soup_htmls:
    body = soup.body
    for tag in body.find_all('li'):
        tag.decompose()
    unlisted_body.append(body.text.strip())

In [12]:
df['non_li_body'] = unlisted_body
df['non_li_body'] 

0       Data Science Manager Data Science Manager Up t...
1       Data Science Engineer Data Science Engineer LO...
2       Head of Data Science Head of Data ScienceLondo...
3       Python Quantitative Researcher - London- Globa...
4       Engineering Lead Engineering Manager | Fin-Tec...
                              ...                        
4303    Software Tester  Telesat (NASDAQ and TSX: TSAT...
4304    Winter Technical Co-op - Cyber Security Engine...
4305    Sr. Architect  Telesat (NASDAQ and TSX: TSAT) ...
4306    Winter Technical Co-op - Systems Integration &...
4307    Senior Director, On-board Software Development...
Name: non_li_body, Length: 4308, dtype: object

7. Exportar datos a archivo json

In [13]:
print(df.head().to_latex(index=False))

\begin{tabular}{llll}
\toprule
                                        title\_html &                                          body\_html &                                            li\_body &                                        non\_li\_body \\
\midrule
           Data Science Manager Leeds - Reed.co.uk &  Data Science Manager Data Science Manager Up t... &  [Leading data science projects under the direc... &  Data Science Manager Data Science Manager Up t... \\
         Data Science Engineer London - Reed.co.uk &  Data Science Engineer Data Science Engineer LO... &  [5 Years' experience implementing within Data ... &  Data Science Engineer Data Science Engineer LO... \\
          Head of Data Science London - Reed.co.uk &  Head of Data Science Head of Data ScienceLondo... &  [Manage a small team of 3 Data Scientists incl... &  Head of Data Science Head of Data ScienceLondo... \\
 Python Quantitative Researcher - London- Globa... &  Python Quantitative Researcher - London- Globa...

In [14]:
#df_jobs.to_csv('jobs.csv',encoding='utf-8',index=False)
df.to_json('jobs_ds.json')